In [3]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


# やりたいこと

## その1
+ 連続する数列の範囲を求める

In [5]:
%%sql
drop table if exists tbl;
create table tbl (
    id integer primary key,
    col1 integer not null
);

insert into tbl
values (1,1),(2,2),(3,3),
    (4,5),(5,7),(6,8),(7,10),(8,11),(9,12);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
9 rows affected.


[]

In [18]:
%%sql
-- lagを使って、diffを求める -> 差が1でなければ足し込んでいって、階段関数にする
select group_id+1 as group_id,
    min(col1) as start_num,
    max(col1) as end_num
from (
    select *,
        -- consecutive_id相当
        sum(case when diff = 1 then 0 else 1 end) over (order by id) as group_id
    from (
        select *,
            coalesce(col1 - lag(col1,1) over (order by id),1) as diff
        from tbl
    ) as tmp1
) as tmp2
group by group_id
order by group_id

*  postgresql://padawan:***@db:5432/dsdojo_db
4 rows affected.


group_id,start_num,end_num
1,1,3
2,5,5
3,7,8
4,10,12


In [33]:
%%sql
-- 差集合: 0 4,6,9 14の間が連続した数列になることを利用
select col+1 as start_num,
    lead(col,1) over (order by col) - 1 as end_num
from (
    select generate_series(
        (select min(col1) from tbl), (select max(col1) from tbl)
    ) as col
    except
    select col1
    from tbl
    union
    values ((select max(col1) from tbl)+1), ((select min(col1) from tbl)-1)
) as tmp


*  postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


start_num,end_num
1,3
5,5
7,8
10,12
14,None


## その2
+ 連続した'o'の範囲を列挙する

In [46]:
%%sql
drop table if exists Seats;
CREATE TABLE Seats
 ( seat   INTEGER NOT NULL  PRIMARY KEY,
   line_id CHAR(1) NOT NULL,
   status CHAR(2) NOT NULL
     CHECK (status IN ('空', '占')) ); 

INSERT INTO Seats VALUES (1, 'A', '占');
INSERT INTO Seats VALUES (2, 'A', '占');
INSERT INTO Seats VALUES (3, 'A', '空');
INSERT INTO Seats VALUES (4, 'A', '空');
INSERT INTO Seats VALUES (5, 'A', '空');
INSERT INTO Seats VALUES (6, 'B', '占');
INSERT INTO Seats VALUES (7, 'B', '占');
INSERT INTO Seats VALUES (8, 'B', '空');
INSERT INTO Seats VALUES (9, 'B', '空');
INSERT INTO Seats VALUES (10,'B', '空');
INSERT INTO Seats VALUES (11,'C', '空');
INSERT INTO Seats VALUES (12,'C', '占');
INSERT INTO Seats VALUES (13,'C', '空');
INSERT INTO Seats VALUES (14,'C', '占');
INSERT INTO Seats VALUES (15,'C', '空');
INSERT INTO Seats VALUES (16,'C', '空');

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [47]:
%%sql
-- lagとcumsumを組み合わせる
select
    min(seat) as start_seat,
    max(seat) as end_seat
from (
    select seat, prev_seat, status,
        -- consecutive_id
        sum(diff) over (order by seat) as group_id
    from (
        select *,
            lag(seat, 1) over (
                order by seat
            ) as prev_seat,
            case when lag(status,1) over (
                order by seat
            ) = status then 0 else 1 end as diff
        from Seats
    ) as tmp1
) as tmp2
group by group_id
having max(status) = '空'
order by group_id

*  postgresql://padawan:***@db:5432/dsdojo_db
4 rows affected.


start_seat,end_seat
3,5
8,11
13,13
15,16


In [62]:
%%sql
-- 3つの自己結合で相関サブクエリ
select min(start_num) as start_num,
    end_num
from (
    select s1.seat as start_num,
        max(s2.seat) as end_num
    from Seats as s1, Seats as s2
    where s1.seat <= s2.seat and
    not exists (
        select *
        from Seats as s3
        where s3.seat between s1.seat and s2.seat and
        s3.status <> '空'
    )
    group by s1.seat
) as tmp
group by end_num

*  postgresql://padawan:***@db:5432/dsdojo_db
4 rows affected.


start_num,end_num
3,5
8,11
13,13
15,16


In [52]:
%%sql
-- 2つの連番を数えるならmaxを使って、すべてのstatusが空であるものを見つける
select seat-1 as start_num,
    seat as end_num
from (
    select *,
        max(status) over (
            order by seat
            rows between 1 preceding and current row
        ) as flag
    from Seats
) as tmp
where flag = '空'

*  postgresql://padawan:***@db:5432/dsdojo_db
6 rows affected.


?column?,seat
3,4
4,5
8,9
9,10
10,11
15,16
